In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions
from Bio.PDB.Polypeptide import one_to_three
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.PDBParser import PDBParser
from pyCodeLib import *
from small_script.myFunctions import *
sys.path.insert(0, "/Users/weilu/openmmawsem")
from helperFunctions.myFunctions import *
from collections import defaultdict
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
frag_name = "/Users/weilu/Research/server/mar_2020/mass_iterative_run/setups/1a1x/1a1x.gro"
frag = pd.read_csv(frag_name, skiprows=2, sep="\s+", header=None, names=["Res_id", "Res", "Type", "i", "x", "y", "z"])

In [ ]:
def get_side_chain_center_of_mass(atoms):
    # ensure complete first
    total = np.array([0., 0., 0.])
    total_mass = 0
    for atom in atoms:
        if atom.get_name() in ["N", "CA", "C", "O", "OXT"]:
            continue
        if atom.element == "H":
            continue
        total += atom.mass * atom.get_coord()
        total_mass += atom.mass
        # print(atom.get_name(), atom.get_coord())
    x_com = total / total_mass
    return x_com

In [3]:
frag

,Res_id,Res,Type,i,x,y,z
0,1,ALA,N,1,0.943,2.721,3.690
1,1,ALA,H,2,1.019,2.802,3.646
2,1,ALA,H2,3,0.840,2.775,3.664
3,1,ALA,H3,4,0.971,2.717,3.806
4,1,ALA,CA,5,0.956,2.591,3.617
5,1,ALA,HA,6,0.924,2.604,3.503
6,1,ALA,C,7,0.877,2.485,3.693
7,1,ALA,O,8,0.927,2.426,3.789
8,1,ALA,CB,9,1.104,2.550,3.606
9,1,ALA,HB1,10,1.163,2.608,3.519
